In [1]:
import os 
os.chdir(os.path.pardir)
# load data from file 
import numpy as np 
save_file_name = ['fea_seq.npy', 'last_observation_seq.npy', 'label_seq.npy', 'masking_seq.npy',
                   'delta_seq.npy', 'train_valid_test_split.npy']
save_folder = 'data/raw/predict-one-day-diff/pol-met-search'
saved_arrays = []
for file_name in save_file_name:
    saved_arrays.append(np.load(os.path.join(save_folder, file_name)))
[fea_seq, last_observation_seq, label_seq, masking_seq, delta_seq, train_valid_test_split] = saved_arrays

# train-test-split 
train_index = [k for k in range(train_valid_test_split[0])]
dev_index = [k for k in range(train_valid_test_split[0], 
                               train_valid_test_split[0] + train_valid_test_split[1])]
test_index = [k for k in range(train_valid_test_split[0] + train_valid_test_split[1],
              train_valid_test_split[0] + train_valid_test_split[1] + train_valid_test_split[2])]

def get_array_by_index_range(nparray_list, label_array, index_range):
    '''
    nparray_list: list of nparrays to select according to index range 
    label_array: select the labels from label array
    '''
    # get non-na index
    non_na_index = []
    for index in index_range:
        if not np.isnan(label_array[index]):
            non_na_index.append(index)
    
    return [k[non_na_index] for k in nparray_list], label_array[non_na_index].reshape(-1)

In [2]:
# normalize delta
# we can normlize this because it's know features 
delta_seq = (delta_seq - np.mean(delta_seq)) / np.std(delta_seq) 

In [3]:
# split set to train, test and dev sets 
# train set
[fea_train, last_train, masking_train, delta_train], label_train =  get_array_by_index_range([fea_seq,last_observation_seq, masking_seq, delta_seq
                                                                 ], label_seq, train_index)
# dev set 
[fea_dev, last_dev, masking_dev, delta_dev], label_dev =  get_array_by_index_range([fea_seq, last_observation_seq, masking_seq, delta_seq
                                                           ], label_seq, dev_index)
# test set 
[fea_test, last_test, masking_test, delta_test], label_test =  get_array_by_index_range([fea_seq, last_observation_seq, masking_seq, delta_seq
                                                              ], label_seq, test_index)

def normalize_feature(fea_train, array_list):
    """
    array_list: [fea_dev, fea_test, last_train, last_dev, last_test] to normalize 
    """
    train_mean = np.nanmean(fea_train, axis=0)
    train_std = np.nanstd(fea_train, axis=0)
    def norm_arr(nparr):
        return(nparr - train_mean)/train_std
    return (norm_arr(fea_train), [norm_arr(k) for k in array_list])

fea_train, [fea_dev, fea_test, last_train, last_dev, last_test] = normalize_feature(fea_train,
                                                                                   [fea_dev, fea_test, 
                                                                                    last_train, last_dev,
                                                                                    last_test])


In [4]:
# record mean after normalization 
x_mean_aft_nor = np.nanmean(fea_train, axis=0)

In [5]:
import torch
import torch.utils.data as utils
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
import random
from torch.autograd import Variable, grad
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math

In [6]:
class FilterLinear(nn.Module):
    def __init__(self, in_features, out_features, filter_square_matrix, bias=True):
        '''
        filter_square_matrix : filter square matrix, whose each elements is 0 or 1.
        '''
        super(FilterLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        use_gpu = torch.cuda.is_available()
        self.filter_square_matrix = None
        if use_gpu:
            self.filter_square_matrix = Variable(filter_square_matrix.cuda(), requires_grad=False)
        else:
            self.filter_square_matrix = Variable(filter_square_matrix, requires_grad=False)
        
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
#         print(self.weight.data)
#         print(self.bias.data)

    def forward(self, input):
#         print(self.filter_square_matrix.mul(self.weight))
        return F.linear(input, self.filter_square_matrix.mul(self.weight), self.bias)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'

In [7]:
# get dataset for grud 
def dataset_aggregation(feature_array, last_obsv, mask, delta):
    # expand dimension of array
    def expd(arr):
        return np.expand_dims(arr, axis=1)
    return np.concatenate((expd(feature_array), expd(last_obsv), expd(mask), expd(delta)), axis = 1)

In [8]:
# dataset_aggregation for train, dev, test 
# train_aggr = dataset_aggregation(fea_train, last_train, masking_train, delta_train)
# pol and met 
train_aggr = dataset_aggregation(last_train[:,:,0:5], 
                                 last_train[:,:,0:5], masking_train[:,:,0:5], delta_train[:,:,0:5])
# dev_aggr = dataset_aggregation(fea_dev, last_dev, masking_dev, delta_dev)
# test_aggr = dataset_aggregation(fea_test, last_test, masking_test, delta_test)
dev_aggr = dataset_aggregation(last_dev[:,:,0:5], 
                               last_dev[:,:,0:5], masking_dev[:,:,0:5], delta_dev[:,:,0:5])
test_aggr = dataset_aggregation(last_test[:,:,0:5], 
                                last_test[:,:,0:5], masking_test[:,:,0:5], delta_test[:,:,0:5])


In [9]:
def train_grud(X_train, y_train, X_valid, y_valid, X_test, y_test, config, x_mean_aft_nor, dropout = 0):
    # no shuffle, keep original order 
    # swap axes for back propagation 
#     def swap_axes(nparr):
#         return nparr.swapaxes(0,1)
#     X_train = swap_axes(X_train)
#     X_valid = swap_axes(X_valid)
#     X_test = swap_axes(X_test)
    
    # model parameters
    input_size = X_train.shape[3]
    h = config["h"]
    t = X_train.shape[2]
    output_dim = 1
    dropout = config["drop"]
    
    model = GRUD(input_size, h, x_mean_aft_nor, output_last = True, dropout=dropout)
    
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    criterion = nn.MSELoss()
    
    device = torch.device('cpu')
    model = model.to(device)
    criterion = criterion.to(device)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=10, factor=0.5, verbose=True)
    
    def train(model, batchsize, X_train, y_train, optimizer, criterion):
        epoch_loss = 0
        model.train()
        total_n = X_train.shape[0]
        num_batches = math.ceil(total_n / batchsize)
        for batch in range(num_batches):
            start = batch*batchsize
            end = (batch+1)*batchsize
            optimizer.zero_grad()
            batch_X = torch.Tensor(X_train[start:end, :])
            batch_y = torch.Tensor(y_train[start:end])
            predictions = model.forward(batch_X).squeeze(1)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / num_batches 
    
    def evaluate(model, X_valid, y_valid, criterion):
        epoch_loss = 0
        model.eval()
        with torch.no_grad():
            batch_X = torch.Tensor(X_valid)
            batch_y = torch.Tensor(y_valid)
            predictions = model.forward(batch_X).squeeze(1)
            epoch_loss = criterion(predictions, batch_y).item()
        return epoch_loss

    def predict(model, X_test):
        epoch_loss = 0
        model.eval()
        with torch.no_grad():
            batch_X = torch.Tensor(X_test)
            predictions = model.forward(batch_X).squeeze(1)
            predictions = predictions.cpu().data.numpy()
        return predictions

    # timing
#     start_time = time.time()
#     predictions = predict(model, X_test)
#     print(predictions.shape)
#     print(predictions)
#     end_time = time.time()
#     print(end_time-start_time)
#     assert False
     
    best_valid = 999999.0
    rand = random.randint(0,100000)
    print('epoch train_loss valid_loss')
    for epoch in range(config["num_epochs"]):
        train_loss = train(model, config["batchsize"], X_train, y_train, optimizer, criterion)
        valid_loss = evaluate(model, X_valid, y_valid, criterion)
        scheduler.step(valid_loss)
        if valid_loss <= best_valid:
            # save model
            best_valid = valid_loss
            print(epoch, train_loss, valid_loss, 'saving model')
            torch.save(model, 'models/lstm_%d.pt' %rand)
        else:
            print(epoch, train_loss, valid_loss)

    model = torch.load('models/lstm_%d.pt' %rand)

    predictions = predict(model, X_test)
    mae = np.mean(np.absolute(predictions-y_test))
    print("mae: ", mae)
    mse = np.mean((predictions - y_test)**2)
    print("mse: ", mse)
#     corr = np.corrcoef(predictions,y_test)[0][1]
#     print("corr: ", corr)
#     true_label = (y_test >= 0)
#     sys.stdout.flush()

In [10]:
train_aggr.shape

(664, 4, 7, 5)

In [11]:
x_mean_aft_nor = np.expand_dims(x_mean_aft_nor, axis=0)

In [12]:
x_mean_aft_nor[:,:,0:5].shape

(1, 7, 5)

In [13]:
class GRUD(nn.Module):
    def __init__(self, input_size, hidden_size, X_mean, output_last = True, dropout=0):
        """
        Recurrent Neural Networks for Multivariate Times Series with Missing Values
        GRU-D: GRU exploit two representations of informative missingness patterns, i.e., masking and time interval.
        cell_size is the size of cell_state.
        
        GRU-D:
            input_size: variable dimension of each time
            hidden_size: dimension of hidden_state
            mask_size: dimension of masking vector
            X_mean: the mean of the historical input data
        """
        
        super(GRUD, self).__init__()
        
        self.hidden_size = hidden_size
        self.delta_size = input_size
        self.mask_size = input_size
        
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            self.identity = torch.eye(input_size).cuda()
            self.zeros = Variable(torch.zeros(input_size).cuda())
            self.X_mean = Variable(torch.Tensor(X_mean).cuda())
        else:
            self.identity = torch.eye(input_size)
            self.zeros = Variable(torch.zeros(input_size))
            self.X_mean = Variable(torch.Tensor(X_mean))
        
        self.zl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size)
        self.rl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size)
        self.hl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size)
        
        self.gamma_x_l = FilterLinear(self.delta_size, self.delta_size, self.identity)
        
        self.gamma_h_l = nn.Linear(self.delta_size, input_size)
        
        self.map_hidden_gamma = nn.Linear(input_size, 1)
        
        self.output_last = output_last
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)
        
    def step(self, x, x_last_obsv, x_mean, h, mask, delta):
        
        batch_size = x.shape[0]
        dim_size = x.shape[1]
        
        delta_x = torch.exp(-torch.max(self.zeros, self.gamma_x_l(delta)))
    
        x = mask * x + (1 - mask) * (delta_x * x_last_obsv + (1 - delta_x) * x_mean)
        
        combined = torch.cat((x, h, mask), 1)
        z = torch.sigmoid(self.zl(combined))
        r = torch.sigmoid(self.rl(combined))
        combined_r = torch.cat((x, r * h, mask), 1)
        h_tilde = torch.tanh(self.hl(combined_r))
        h = (1 - z) * h + z * h_tilde
        
        return h
    
    def forward(self, input):
        batch_size = input.size(0)
        type_size = input.size(1)
        step_size = input.size(2)
        spatial_size = input.size(3)
        
        Hidden_State = self.initHidden(batch_size)
        
        def squeeze_d1(matrix):
            return torch.squeeze(matrix, dim=1)
        X = squeeze_d1(input[:,0,:,:])
        X_last_obsv = squeeze_d1(input[:,1,:,:])
        Mask = squeeze_d1(input[:,2,:,:])
        Delta = squeeze_d1(input[:,3,:,:])
        
        outputs = None
        for i in range(step_size):
            Hidden_State = self.step(squeeze_d1(X[:,i:i+1,:])\
                                     , squeeze_d1(X_last_obsv[:,i:i+1,:])\
                                     , squeeze_d1(self.X_mean[:,i:i+1,:])\
                                     , Hidden_State\
                                     , squeeze_d1(Mask[:,i:i+1,:])\
                                     , squeeze_d1(Delta[:,i:i+1,:]))
            if outputs is None:
                outputs = Hidden_State.unsqueeze(1)
            else:
                outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
                
        if self.output_last:
            last_hs = outputs[:,-1,:]
            output = F.relu(self.fc1(last_hs))
            output = self.dropout(output)
            output = self.fc2(output)
            return output
        else:
            raise Exception("Not output last")

    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State

In [14]:
# pol and met # structure one
config = {'h':128, 'lr':0.001, 'num_epochs':100, 'batchsize':32, 'drop':0.5}
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_grud(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, config, x_mean_aft_nor[:,:,0:5])

epoch train_loss valid_loss
0 39.1354892821539 27.73879623413086 saving model


/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type GRUD. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type FilterLinear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


1 31.859557832990372 20.308773040771484 saving model
2 26.215213139851887 19.68356704711914 saving model
3 26.5877141498384 18.73686981201172 saving model
4 24.221855663117907 17.836490631103516 saving model
5 23.8710511525472 17.857942581176758
6 22.695493107750302 17.79047393798828 saving model
7 22.450421106247674 17.765544891357422 saving model
8 22.2658657346453 17.71568489074707 saving model
9 22.250603176298597 17.537153244018555 saving model
10 21.92252613249279 17.35944175720215 saving model
11 21.75543685186477 17.124847412109375 saving model
12 21.758784521193732 17.182846069335938
13 20.843877065749396 17.347431182861328
14 21.440978776840936 16.955883026123047 saving model
15 20.711157844180153 17.308639526367188
16 20.059633255004883 17.224584579467773
17 19.68612779889788 17.209074020385742
18 19.91121024177188 16.88689422607422 saving model
19 19.52487591334752 17.322040557861328
20 18.648843401954288 17.19211769104004
21 19.16448488689604 17.01597785949707
22 18.510307

In [17]:
# pol and met 
config = {'h':128, 'lr':0.001, 'num_epochs':100, 'batchsize':32, 'drop':0.5}
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_grud(train_aggr, label_train, dev_aggr, label_dev, test_aggr, label_test, config, x_mean_aft_nor[:,:,0:5])

epoch train_loss valid_loss
0 39.67365242186047 29.49156379699707 saving model


/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type GRUD. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/flynn/Documents/Desktop/softs/miniconda/envs/airpol/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type FilterLinear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


1 33.212131727309455 19.959796905517578 saving model
2 27.969510078430176 17.87363052368164 saving model
3 26.164809408641997 18.966472625732422
4 24.425521123976935 17.514774322509766 saving model
5 24.32747404915946 17.448461532592773 saving model
6 22.880385943821498 17.30282974243164 saving model
7 22.904385930015927 17.016151428222656 saving model
8 22.659231458391464 16.945783615112305 saving model
9 22.605987412588938 16.628305435180664 saving model
10 22.281026386079333 16.504920959472656 saving model
11 22.29319890340169 16.37874984741211 saving model
12 22.12033766791934 16.241493225097656 saving model
13 21.247778165908088 16.185623168945312 saving model
14 21.973103841145832 16.27969741821289
15 21.031504767281668 16.048723220825195 saving model
16 20.93706871214367 16.229759216308594
17 20.408146358671644 16.00263023376465 saving model
18 20.87461235409691 15.986905097961426 saving model
19 20.44986997331892 16.121902465820312
20 19.544604573931014 16.23101234436035
21 20.